In [ ]:
import os
import shutil
import pandas as pd
import numpy as np
import datetime as dt
import pickle
from scipy import stats
import matplotlib.pyplot as plt
from matplotlib import style
from IPython.display import Audio, display

from sklearn.svm import SVC
from sklearn import model_selection
from sklearn.datasets import make_regression
from sklearn.model_selection import cross_val_score
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn.preprocessing import PolynomialFeatures
from sklearn.metrics import mean_squared_error
from sklearn.linear_model import LinearRegression
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import chi2
from sklearn.ensemble import ExtraTreesClassifier
from pandas.plotting import autocorrelation_plot
pd.options.mode.chained_assignment = None   # turn off warnings

style.use('ggplot')

# === check size 
def checksize(dataframe):
    q = dataframe.shape[1]
    o = dataframe.shape[0]
    print('data rows: %d' % o)
    print('data columns: %d' % q)
    print(dataframe.head(2))
    print(dataframe.dtypes)
    
# === output folder
newpath = r'./output/'
if os.path.exists(newpath):
    shutil.rmtree(newpath, ignore_errors=True)
os.makedirs(newpath)

## get delay data from .csv

In [ ]:
# df69 = pd.read_csv(r'./delay_data/df_69.csv')
df = pd.read_csv(r'./delay_data/df.csv')

# df69['time'] = pd.to_datetime(df69['time'])
df['time'] = pd.to_datetime(df['time'])

# checksize(df)

## get weather data from .csv

In [ ]:
# === input folder
path = r'./weather/*.csv'
# === import csv as dataframe
new_cols = ['weather']
we = pd.read_csv('./weather/agrometeo-data.csv', encoding='Latin-1', header=None, names=new_cols)
wet1 = pd.DataFrame(data=we)
# === clean-up
wet = wet1.iloc[3:]
wet.loc[:, 'time'] = wet.weather.str.split(';').str.get(0)
wet.loc[:, 'temp'] = wet.weather.str.split(';').str.get(1)
wet.loc[:, 'rain'] = wet.weather.str.split(';').str.get(2)
wet.drop(wet.columns[[0]], axis=1, inplace=True)
wet.loc[:, 'time'] = pd.to_datetime(wet.loc[:, 'time'])
wet.loc[:, 'rain'] = wet.loc[:, 'rain'].apply(pd.to_numeric, errors='coerce')
wet['temp'] = wet['temp'].astype(float)
wet = wet.dropna(how='any')

# === 2D temp and rain
x = wet['temp']
y = wet['rain']
plt.scatter(x, y)
plt.title("")
plt.xlabel("temp[C]")
plt.ylabel("precipitation[mm/m2]")
plt.savefig('./output/raintemp.png', format='png', dpi=800)

## merge delay and weather

In [ ]:
# === inner merge on both time col which are in datetime format
# === choose between df and df69!!!
merge = df.merge(wet, left_on='time', right_on='time', how='inner')

checksize(merge)

# # === 2D temp and rain
# x = merge['rain']
# y = merge['diff']
# plt.scatter(x, y)
# plt.title("")
# plt.xlabel("precipitation[mm/m2]")
# plt.ylabel("delay[min]")
# plt.savefig('./output/raintemp.png', format='png', dpi=800)

## smooth out data

In [ ]:
smooth = merge.copy()
# smooth = smooth.set_index('time')
diffmean = smooth.groupby('time').diff.mean().to_frame()
rainmean = smooth.groupby('time').rain.mean().to_frame()
weekday = smooth.groupby('time').weekday.mean().to_frame()
hour = smooth.groupby('time').hour.mean().to_frame()
tempmean = smooth.groupby('time').temp.mean().to_frame()

smooth = diffmean.merge(rainmean, left_index=True, right_index=True, how='left')
smooth = smooth.merge(weekday, left_index=True, right_index=True, how='left')
smooth = smooth.merge(hour, left_index=True, right_index=True, how='left')
smooth = smooth.merge(tempmean, left_index=True, right_index=True, how='left')
smooth = smooth.reset_index()


# === Changes in dataset
# smooth = smooth[smooth['diff'] > 0]    # only positive delays
# smooth = smooth[smooth['temp'] > 0]
# smooth['diff'] = smooth['diff']*60    # seconds delay
# smooth = smooth[smooth['diff'] > 120]    # focus on extreme delays
# smooth['diff'] = smooth['diff'].astype('int')
# smooth['temp'] = smooth['temp'].astype('int')
# smooth['rain'] = smooth['rain']*10     # from mm/m2 to (mm/m2)/10
# smooth['rain'] = smooth['rain'].astype('int')

q = smooth['diff'].quantile(0.99)
print(q)
smooth = smooth[smooth['diff'] <= q]
smooth = smooth[smooth['diff'] >= -q]
smooth = smooth.dropna(how='any')

smooth['temp'] = smooth['temp'].round(decimals=2)
smooth['rain'] = smooth['rain'].round(decimals=2)

dataframe = smooth
array = dataframe.values

# Weekday polynomial representation

In [ ]:
print('[time(0), diff(1), rain(2), weekday(3), hour(4), temp(5)]')
X = array[:,[3]]
y = X**4 + X**3 + X + 1

x_train, x_test, y_train, y_test = model_selection.train_test_split(X, y, test_size=0.3, random_state=42)

rmses = []
degrees = np.arange(1, 10)
min_rmse, min_deg = 1e10, 0

for deg in degrees:

    # Train features
    poly_features = PolynomialFeatures(degree=deg, include_bias=False)
    x_poly_train = poly_features.fit_transform(x_train)

    # Linear regression
    poly_reg = LinearRegression()
    poly_reg.fit(x_poly_train, y_train)

    # Compare with test data
    x_poly_test = poly_features.fit_transform(x_test)
    poly_predict = poly_reg.predict(x_poly_test)
    poly_mse = mean_squared_error(y_test, poly_predict)
    poly_rmse = np.sqrt(poly_mse)
    rmses.append(poly_rmse)

    # Cross-validation of degree
    if min_rmse > poly_rmse:
        min_rmse = poly_rmse
        min_deg = deg

# Plot and present results
print('Best degree {} with RMSE {}'.format(min_deg, min_rmse))

x = smooth['weekday']
y = smooth['diff']

z = np.polyfit(x, y, min_deg)
p = np.poly1d(z)

xp = np.linspace(0, 6, 100)
_ = plt.plot(x, y, '.', xp, p(xp), '-')
plt.title("Weekday")
plt.xlabel("weekday")
plt.ylabel("delay[minutes]")
plt.savefig('./output/weekday_poly.png', format='png', dpi=1200)
plt.show()


# plt.plot(smooth['weekday'], smooth['diff'], '.')
# plt.plot(smooth['weekday'], slope*smooth['weekday'] + intercept, '-')
# plt.xlabel("weekday")
# plt.ylabel("delay[seconds]")
# plt.savefig('./output/weekday_delayLinReg.png', format='png', dpi=800)
# plt.show()

## hour polynomial representation

In [ ]:
print('[time(0), diff(1), rain(2), weekday(3), hour(4), temp(5)]')

X = array[:,[4]]
y = X**4 + X**3 + X + 1

x_train, x_test, y_train, y_test = model_selection.train_test_split(X, y, test_size=0.3, random_state=42)

rmses = []
degrees = np.arange(1, 30)
min_rmse, min_deg = 1e10, 0

for deg in degrees:

    # Train features
    poly_features = PolynomialFeatures(degree=deg, include_bias=False)
    x_poly_train = poly_features.fit_transform(x_train)

    # Linear regression
    poly_reg = LinearRegression()
    poly_reg.fit(x_poly_train, y_train)

    # Compare with test data
    x_poly_test = poly_features.fit_transform(x_test)
    poly_predict = poly_reg.predict(x_poly_test)
    poly_mse = mean_squared_error(y_test, poly_predict)
    poly_rmse = np.sqrt(poly_mse)
    rmses.append(poly_rmse)

    # Cross-validation of degree
    if min_rmse > poly_rmse:
        min_rmse = poly_rmse
        min_deg = deg

# Plot and present results
print('Best degree {} with RMSE {}'.format(min_deg, min_rmse))

x = smooth['hour']
y = smooth['diff']

z = np.polyfit(x, y, min_deg)
p = np.poly1d(z)

xp = np.linspace(0, 23, 100)
_ = plt.plot(x, y, '.', xp, p(xp), '-')
plt.title("Hour of Day")
plt.xlabel("hour of day")
plt.ylabel("delay[minutes]")
plt.savefig('./output/hour_poly.png', format='png', dpi=1200)
plt.show()

autocorrelation_plot([x, y])
plt.show()


## Linear regressions

In [ ]:
# === temp
print('correlation temp-delay: ', np.corrcoef(smooth.loc[:, 'temp'], smooth.loc[:, 'diff']))
slope, intercept, r_value, p_value, std_err = stats.linregress(smooth.loc[:, 'temp'], smooth.loc[:, 'diff'])
print("r-squared value between temperature and delay:", r_value ** 2)

plt.plot(smooth['temp'], smooth['diff'], '.')
plt.plot(smooth['temp'], slope*smooth['temp'] + intercept, '-')
plt.title("Temperature")
plt.xlabel("temp[C]")
plt.ylabel("delay[minutes]")
plt.savefig('./output/temp_lin.png', format='png', dpi=1200)
plt.show()

scurr = smooth[smooth['rain'] > 0]    # only rainevents
# === rain
print('correlation rain-delay: ', np.corrcoef(scurr['rain'], scurr['diff']))
slope, intercept, r_value, p_value, std_err = stats.linregress(scurr.loc[:, 'rain'], scurr.loc[:, 'diff'])
print('slope, intercept, r_value, p_value, std_err :', slope, intercept, r_value, p_value, std_err)
print("r-squared value between rain and delay:", r_value ** 2)
plt.plot(scurr['rain'], scurr['diff'], '.')
plt.plot(scurr['rain'], slope*scurr['rain'] + intercept, '-')
plt.title("Rain")
plt.xlabel("rain[mm/h]")
plt.ylabel("delay[minutes]")
plt.savefig('./output/rain_lin.png', format='png', dpi=1200)
plt.show()


# x1 = X_train[:,1].astype(int)
# x2 = X_test[:,1].astype(int)

## preprocessing

In [ ]:
print('[time(0), diff(1), rain(2), weekday(3), hour(4), temp(5)]')

scurr['diff'] = scurr['diff']*60    # seconds delay
dataframe = scurr
array = dataframe.values

X = array[:,[2,3,4,5]]
Y = array[:,[1]].astype('int')
Y = Y.ravel()

test_size = 0.33
seed = 42
X_train, X_test, Y_train, Y_test = model_selection.train_test_split(X, Y, test_size=test_size, random_state=seed)

# sorted_index = np.argsort(X_test)
# X_test = X_test[sorted_index]
# Y_test = Y_test[sorted_index]

## Automatic Feature Extraction

In [ ]:
print('features: rain(0), weekday(1), hour(2), temp(3)')
# # === chi2 feature extraction  # not possible with negative values
# test = SelectKBest(score_func=chi2, k=4)
# fit = test.fit(X_train, Y_train)
# # summarize scores
# np.set_printoptions(precision=3)
# print('chi2 scores: ', fit.scores_)
# features = fit.transform(X)
# # summarize selected features
# print('chi2 features: ',features[0:5,:])

# === ExtraTreesClassifier feature extraction
model = ExtraTreesClassifier()
model.fit(X, Y)
print('ExtraTreesClassifier feature importance: ', model.feature_importances_)

# === PCA
features = ['rain','weekday', 'hour', 'temp']
# Separating out the features
x = smooth.loc[:, features].values
# Separating out the target
y = smooth.loc[:,['diff']].values
# Standardizing the features
x = StandardScaler().fit_transform(x)

# feature extraction
pca = PCA(n_components=2)
fit = pca.fit(x)
# summarize components
print("Explained Variance: ", fit.explained_variance_ratio_)
print('PCA components: ', fit.components_)



## ARIMA

In [ ]:
print('[time(0), diff(1), rain(2), weekday(3), hour(4), temp(5)]')
from statsmodels.tsa.arima_model import ARIMA
series = array[:,[0,1]]
series[:,0] = smooth['time'].dt.strptime('190'+x, '%Y-%m-%d %h')
# series[:,0] = np.array(series[:,0], dtype='datetime64[h]')
series[:,1] = np.array(series[:,1], dtype='int')

print(series)
# print(series.dtypes)
# fit model
model = ARIMA(series, order=(5,1,0))
model_fit = model.fit(disp=0)
print(model_fit.summary())
# plot residual errors
residuals = pd.DataFrame(model_fit.resid)
residuals.plot()
plt.show()
residuals.plot(kind='kde')
plt.show()
print(residuals.describe())

## Selected Features

In [ ]:
print('[time(0), diff(1), rain(2), weekday(3), hour(4), temp(5)]')

dataframe = scurr
array = dataframe.values
X = array[:,[2,3,4,5]]
Y = array[:,[1]].astype('int')
Y = Y.ravel()

test_size = 0.33
seed = 42
X_train, X_test, Y_train, Y_test = model_selection.train_test_split(X, Y, test_size=test_size, random_state=seed)

## LogisticRegression

In [ ]:
from sklearn.linear_model import LogisticRegression

model = LogisticRegression()
model.fit(X, Y)

scores = cross_val_score(model, X, Y, scoring='neg_mean_squared_error')
print('neg_mean_squared_error: ', scores.mean())

# # save the model to disk
# filename = 'first_prediction.sav'
# pickle.dump(model, open(filename, 'wb'))

# # load the model from disk
# loaded_model = pickle.load(open(filename, 'rb'))
# result = loaded_model.score(X_test, Y_test)
# print(result)

pred = model.predict(X_test)
print('features: [rain, weekday, hour, temp]')
for i in range(10):
    print("X=%s, Predicted=%s" % (X_test[i] , Y_test[i]))

## Multi-Layer Perceptron

In [ ]:
from sklearn.neural_network import MLPClassifier

clf = MLPClassifier(solver='adam', alpha=1e-5, hidden_layer_sizes=(10, 2), random_state=1,
                    max_iter=300, shuffle=True, momentum=0.8, epsilon=1e-08, verbose=True, early_stopping=True)
clf.fit(X_train, Y_train)

scores = cross_val_score(model, X_train, Y_train, scoring='neg_mean_squared_error')
print('neg_mean_squared_error:', scores.mean())

pred = model.predict(X_test)
print('[weekday, hour]')
for i in range(10):
    print("X=%s, Predicted=%s" % (X_test[i], Y_test[i]))

display(Audio("./weather/ape.mp3", autoplay=True))

## visualize, analyse data

In [ ]:
from mpl_toolkits.mplot3d import Axes3D

style.use("ggplot")

# show input features
plt.scatter(X[:, 0], X[:, 3], s=40, c=Y, cmap=plt.cm.Spectral)
plt.xlabel('weekday')
plt.ylabel('precipitation[mm*10/m2]')
plt.title('precipitation & weekday')
plt.savefig('./output/weekdays&rain.png', format='png', dpi=800)
plt.show()

month = scurr.copy()
day = scurr.copy()
hour = smooth.copy()

month['time'] = month.loc[:, 'time'].apply(lambda x: x.month)
day['time'] = day.loc[:, 'time'].apply(lambda x: x.day)
hour['time'] = hour.loc[:, 'time'].apply(lambda x: x.hour)

# # === 2D monthly distribution of delays
# x = month.loc[:, 'time']
# y = month.loc[:, 'diff']
# plt.scatter(x, y)
# plt.title("monthly distribution")
# plt.xlabel("months")
# plt.ylabel("delay[minutes]")
# plt.savefig('./output/2Dmonthly.png', format='png', dpi=800)
# # plt.show()

# # === 2D daily distribution of delays
# x = day.loc[:, 'weekday']
# y = day.loc[:, 'diff']
# plt.scatter(x, y)
# plt.title("daily distribution")
# plt.xlabel("weekday")
# plt.ylabel("delay[minutes]")
# plt.savefig('./output/2Ddaily.png', format='png', dpi=800)
# # plt.show()

# # === 2D hourly distribution of delays
# x = hour.loc[:, 'time']
# y = hour.loc[:, 'diff']
# plt.scatter(x, y)
# plt.title("hourly distribution")
# plt.xlabel("hours")
# plt.ylabel("delay[minutes]")
# plt.savefig('./output/2Dhourly.png', format='png', dpi=800)
# # plt.show()

# # === 2D dependency on rain
# x = smooth.loc[:, 'rain']
# y = smooth.loc[:, 'diff']
# plt.scatter(x, y)
# plt.title("correlation")
# plt.xlabel("rain[mm/m2]")
# plt.ylabel("delay[minutes]")
# plt.savefig('./output/2Drain-delay.png', format='png', dpi=800)
# # plt.show()

# === 3D Figure rain, time, delay
x = hour.loc[:, 'time']
y = hour.loc[:, 'rain']
z = hour.loc[:, 'diff']
fig = plt.figure()
ax = Axes3D(fig)
ax.scatter(x, y, z)
plt.title("Correlation")
ax.set_xlabel('hours')
ax.set_ylabel('rain [mm/m2]')
ax.set_zlabel('delay[minutes]')
plt.savefig('./output/3DcorrelationHourly.png', format='png', dpi=800)
# plt.show()

# === other statistics on data

print(smooth.loc[:,'rain'].describe())
print(smooth.loc[:,'diff'].describe())

slope, intercept, r_value, p_value, std_err = stats.linregress(smooth.loc[:, 'rain'], smooth.loc[:, 'diff'])
print("r-squared value between rain and delay:", r_value ** 2)

display(Audio("./weather/ape.mp3", autoplay=True))